### Phenotype Data

In [ ]:
import pandas
import os

# This query represents dataset "srWGS_all_selected_traits" for domain "person" and was generated for All of Us Controlled Tier Dataset v7
dataset_95945546_person_sql = """
    SELECT
        person.person_id,
        person.gender_concept_id,
        p_gender_concept.concept_name as gender,
        person.birth_datetime as date_of_birth,
        person.race_concept_id,
        p_race_concept.concept_name as race,
        person.ethnicity_concept_id,
        p_ethnicity_concept.concept_name as ethnicity,
        person.sex_at_birth_concept_id,
        p_sex_at_birth_concept.concept_name as sex_at_birth 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.person` person 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_gender_concept 
            ON person.gender_concept_id = p_gender_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_race_concept 
            ON person.race_concept_id = p_race_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_ethnicity_concept 
            ON person.ethnicity_concept_id = p_ethnicity_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_sex_at_birth_concept 
            ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id  
    WHERE
        person.PERSON_ID IN (SELECT
            distinct person_id  
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
        WHERE
            cb_search_person.person_id IN (SELECT
                person_id 
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
            WHERE
                has_whole_genome_variant = 1 ) )"""

dataset_95945546_person_df = pandas.read_gbq(
    dataset_95945546_person_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_95945546_person_df.head(5)

In [ ]:
import os

In [ ]:
os.environ['WORKSPACE_BUCKET']

In [ ]:
dataset_95945546_person_df.to_csv('Demographics.tsv', sep='\t', index=False, header=True)

In [ ]:
import pandas
import os

# This query represents dataset "srWGS_all_selected_traits" for domain "survey" and was generated for All of Us Controlled Tier Dataset v7
dataset_95945546_survey_sql = """
    SELECT
        answer.person_id,
        answer.survey_datetime,
        answer.survey,
        answer.question_concept_id,
        answer.question,
        answer.answer_concept_id,
        answer.answer,
        answer.survey_version_concept_id,
        answer.survey_version_name  
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.ds_survey` answer   
    WHERE
        (
            question_concept_id IN (1585940) 
            OR question_concept_id IN (SELECT
                DISTINCT concept_id                         
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c                         
            JOIN
                (SELECT
                    CAST(cr.id as string) AS id                               
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr                               
                WHERE
                    concept_id IN (1586134)                               
                    AND domain_id = 'SURVEY') a 
                    ON (c.path like CONCAT('%', a.id, '.%'))                         
            WHERE
                domain_id = 'SURVEY'                         
                AND type = 'PPI'                         
                AND subtype = 'QUESTION')
        )  
        AND (
            answer.PERSON_ID IN (SELECT
                distinct person_id  
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
            WHERE
                cb_search_person.person_id IN (SELECT
                    person_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                WHERE
                    has_whole_genome_variant = 1 ) )
        )"""

dataset_95945546_survey_df = pandas.read_gbq(
    dataset_95945546_survey_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_95945546_survey_df.head(5)

In [ ]:
dataset_95945546_survey_df.to_csv('Survey.tsv', sep='\t', index=False, header=True)

In [ ]:
import pandas
import os

# This query represents dataset "srWGS_all_selected_traits" for domain "measurement" and was generated for All of Us Controlled Tier Dataset v7
dataset_95945546_measurement_sql = """
    SELECT
        measurement.person_id,
        measurement.measurement_concept_id,
        m_standard_concept.concept_name as standard_concept_name,
        m_standard_concept.concept_code as standard_concept_code,
        m_standard_concept.vocabulary_id as standard_vocabulary,
        measurement.measurement_datetime,
        measurement.measurement_type_concept_id,
        m_type.concept_name as measurement_type_concept_name,
        measurement.operator_concept_id,
        m_operator.concept_name as operator_concept_name,
        measurement.value_as_number,
        measurement.value_as_concept_id,
        m_value.concept_name as value_as_concept_name,
        measurement.unit_concept_id,
        m_unit.concept_name as unit_concept_name,
        measurement.range_low,
        measurement.range_high,
        measurement.visit_occurrence_id,
        m_visit.concept_name as visit_occurrence_concept_name,
        measurement.measurement_source_value,
        measurement.measurement_source_concept_id,
        m_source_concept.concept_name as source_concept_name,
        m_source_concept.concept_code as source_concept_code,
        m_source_concept.vocabulary_id as source_vocabulary,
        measurement.unit_source_value,
        measurement.value_source_value 
    FROM
        ( SELECT
            * 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.measurement` measurement 
        WHERE
            (
                measurement_concept_id IN (SELECT
                    DISTINCT c.concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                JOIN
                    (SELECT
                        CAST(cr.id as string) AS id       
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                    WHERE
                        concept_id IN (3019171, 3023540, 3036277, 3038553, 4030731)       
                        AND full_text LIKE '%_rank1]%'      ) a 
                        ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                        OR c.path LIKE CONCAT('%.', a.id) 
                        OR c.path LIKE CONCAT(a.id, '.%') 
                        OR c.path = a.id) 
                WHERE
                    is_standard = 1 
                    AND is_selectable = 1) 
                OR  measurement_source_concept_id IN (SELECT
                    DISTINCT c.concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                JOIN
                    (SELECT
                        CAST(cr.id as string) AS id       
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                    WHERE
                        concept_id IN (903124, 903127, 903128, 903133, 903134, 903135)       
                        AND full_text LIKE '%_rank1]%'      ) a 
                        ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                        OR c.path LIKE CONCAT('%.', a.id) 
                        OR c.path LIKE CONCAT(a.id, '.%') 
                        OR c.path = a.id) 
                WHERE
                    is_standard = 0 
                    AND is_selectable = 1)
            )  
            AND (
                measurement.PERSON_ID IN (SELECT
                    distinct person_id  
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        has_whole_genome_variant = 1 ) )
            )) measurement 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` m_standard_concept 
            ON measurement.measurement_concept_id = m_standard_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` m_type 
            ON measurement.measurement_type_concept_id = m_type.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` m_operator 
            ON measurement.operator_concept_id = m_operator.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` m_value 
            ON measurement.value_as_concept_id = m_value.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` m_unit 
            ON measurement.unit_concept_id = m_unit.concept_id 
    LEFT JOIn
        `""" + os.environ["WORKSPACE_CDR"] + """.visit_occurrence` v 
            ON measurement.visit_occurrence_id = v.visit_occurrence_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` m_visit 
            ON v.visit_concept_id = m_visit.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` m_source_concept 
            ON measurement.measurement_source_concept_id = m_source_concept.concept_id"""

dataset_95945546_measurement_df = pandas.read_gbq(
    dataset_95945546_measurement_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_95945546_measurement_df.head(5)

In [ ]:
dataset_95945546_measurement_df.to_csv('pheno_raw/measurements.tsv', sep='\t', index=False, header=True)

### Reference Genotype 1KG & HGDP

In [ ]:
#! mkdir -p genotype/1KGPHGDP
! mkdir -p genotype/AllofUS

In [ ]:
%%bash

#Download the data from the google bucket. This will take a while.
gsutil cp gs://fc-secure-47ccf5a8-b9ba-460a-aa03-dea8d260953b/Data/1KGPHGDP.tar.gz genotype/

In [ ]:
%%bash

#Un-TAR the data.
tar -xvf genotype/1KGPHGDP.tar.gz --directory genotype/

In [ ]:
%%bash 

#Add the prefix "chr" to the bim file to match with how All of Us supplies the data.
awk '{print "chr"$1"\t"$2"\t"$3"\t"$4"\t"$5"\t"$6}' genotype/1KGPHGDP/extractedChrAllUnpruned.bim > genotype/1KGPHGDP/extractedChrAllUnpruned.Prefix.bim

In [ ]:
%%bash 

mv genotype/1KGPHGDP/extractedChrAllUnpruned.bim genotype/1KGPHGDP/extractedChrAllUnpruned.Original.bim
mv genotype/1KGPHGDP/extractedChrAllUnpruned.Prefix.bim genotype/1KGPHGDP/extractedChrAllUnpruned.bim

In [ ]:
%%bash

#Split the whole dataset into 22 different file sets. This will allow us to run processes on 22 parallel cores.
seq 1 22 | xargs -I {} -P 22 sh -c "plink --bfile genotype/1KGPHGDP/extractedChrAllUnpruned --keep-allele-order --allow-no-sex --chr {} --make-bed --out genotype/1KGPHGDP/extractedChr{}"

### AllofUS Genotype Data

In [ ]:
import os

genomic_location = os.getenv("CDR_STORAGE_PATH")
print(genomic_location)

In [ ]:
%%bash

#Look at the files in google bucket.
gsutil -u $GOOGLE_PROJECT ls -lh gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/acaf_threshold_v7.1/plink_bed

In [ ]:
%%bash

#Copy the BED files from google bucket - this will take a while.
gsutil -m -u $GOOGLE_PROJECT cp -r gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/acaf_threshold_v7.1/plink_bed/acaf_threshold.chr21.* genotype/AllofUS/

In [ ]:
%%bash

gsutil -m -u $GOOGLE_PROJECT cp gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/aux/relatedness/relatedness_flagged_samples.tsv .

In [ ]:
%%bash

gsutil -m -u $GOOGLE_PROJECT cp gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/aux/qc/flagged_samples.tsv .

In [ ]:
import numpy as np
np.arange(1, 21)

In [ ]:
import subprocess
import os

In [ ]:
for i in np.arange(1, 21):
    ### Download genotype data
    subprocess.run([
        'gsutil', '-m', '-u', os.environ['GOOGLE_PROJECT'], 
        'cp', '-r', 
        'gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/acaf_threshold_v7.1/plink_bed/acaf_threshold.chr' + str(i) + '.*', 
        'genotype/AllofUS/'
    ])
    
    ### Extract by population and basic QC
    subprocess.run([
        'plink',
        '--bfile', 'genotype/AllofUS/acaf_threshold.chr' + str(i),
        '--keep', 'genotype/Afr_IIDs.tsv',
        '--no-sex',
        '--maf', '0.05',
        '--indep-pairwise', '50', '5', '0.2',
        '--hwe', '1e-6',
        '--make-bed',
        '--out', 'genotype/AllofUS/Afr/acaf_threshold.chr' + str(i) + '.pruned'
    ])
    
    ### Remove the downloaded files to save space
    subprocess.run(['rm', 'genotype/AllofUS/acaf_threshold.chr' + str(i) + '.*'])

In [ ]:
gsutil -m -u $GOOGLE_PROJECT cp -r gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/acaf_threshold_v7.1/plink_bed/acaf_threshold.chr21.* genotype/AllofUS/